In [1]:
import re #importação do regex
# Leitura do arquivo csv
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None) #Não limita a exibição dos n°s de registros
pd.options.display.max_columns = None #Não limita a exibição das colunas dos DataFrames
pd.set_option('display.max_colwidth', None) #Não  limita a exibição do conteúdo das células
df_cat_foto = pd.read_csv("fotografia_tabela49.csv", low_memory=False)
df_bd_foto = pd.read_csv("fotografia_geralBD.csv", low_memory=False)
df_papel_foto = pd.read_csv("papelfotografico.csv", low_memory=False)

In [2]:
print('cat', len(df_cat_foto))
print('bd', len(df_bd_foto))
print('papel foto', len(df_papel_foto))

cat 211
bd 209
papel foto 16


In [4]:
len(df_bd_foto) - len(df_cat_foto[df_cat_foto['nreferencia'].isin(df_bd_foto.nreferencia.values.tolist())])

1

Apenas 1 registro de bd não está em cat

In [5]:
df_bd_foto[~df_bd_foto['nreferencia'].isin(df_cat_foto.nreferencia.values.tolist())]

,nreferencia,especificação,numero,ligação,sistema,titulo,local,data,produção
200,NaN,Professor Emérito. Concessão do Título de Professor Emérito a Osvaldo Porchat pela Congregação da Faculdade de Filosofia.,137.0,5730,NaN,Professor Emérito,NaN,NaN,NaN


In [6]:
#É um em que a referencia é vazia, separar para análise
csv_exception = df_bd_foto[~df_bd_foto['nreferencia'].isin(df_cat_foto.nreferencia.values.tolist())]
csv_exception.insert(len(csv_exception.columns),'origem', ['GeralBD'], True)
csv_exception

,nreferencia,especificação,numero,ligação,sistema,titulo,local,data,produção,origem
200,NaN,Professor Emérito. Concessão do Título de Professor Emérito a Osvaldo Porchat pela Congregação da Faculdade de Filosofia.,137.0,5730,NaN,Professor Emérito,NaN,NaN,NaN,GeralBD


Vamos apagar este registro bd

In [7]:
df_bd_foto = df_bd_foto.drop(200)

In [9]:
len(df_cat_foto[df_cat_foto['nreferencia'].isin(df_bd_foto.nreferencia.values.tolist())]) == len(df_bd_foto)
#agora todas as nreferencias de bd têm em cat

True

In [10]:
def compare_df(df1, df2, name_id, col_name,df1_name="df1",df2_name="df2"):
    """
    Esta função retorna um dataframe com registros que possuem o mesmo name_id mas campos (col_name) diferentes.
    df1_name se refere ao titulo da tebela, que irá aparecer no campo origem nos registros
    """
    id1 =  set(df1[name_id])
    id2 =  set(df2[name_id])
    id_commom = list(id1.intersection(id2))
    df_structure = {}
    df_structure[name_id] = []
    for col in col_name:
        df_structure[col] = []
    df_structure['origem'] = []
    for i in id_commom:
        for col in col_name:
            if(
            len(df1[df1[name_id] == i][col]) > 0
                and
            len(df2[df1[name_id] == i][col]) > 0
            ):
                aux1 = df1[df1[name_id] == i][col].item()
                aux2 = df2[df2[name_id] == i][col].item()
                if(str(aux1) != 'nan'  or str(aux2) != 'nan'):
                    if (aux1 != aux2):
                        for c in col_name:
                            aux = df_structure[c]
                            aux.append(df1[df1[name_id] == i][c].item())
                            df_structure[c] = aux
                        aux = df_structure[name_id]
                        aux.append(i)
                        df_structure[name_id] = aux
                        aux = df_structure['origem']
                        aux.append(df1_name)
                        df_structure['origem'] = aux
                        for c in col_name:
                            aux = df_structure[c]
                            aux.append(df2[df2[name_id] == i][c].item())
                            df_structure[c] = aux
                        aux = df_structure[name_id]
                        aux.append(i)
                        df_structure[name_id] = aux
                        aux = df_structure['origem']
                        aux.append(df2_name)
                        df_structure['origem'] = aux
                        break
    return pd.DataFrame(df_structure)

In [11]:
#pego os valores diferentes entre df_cat_foto e df_bd_foto para análise
result = compare_df(df_cat_foto,df_bd_foto,'nreferencia',['especificação','numero','local', 'titulo', 'produção', 'data', 'sistema'],df1_name="Tabela49",df2_name="GeralBD")

<ipython-input-10-5a7f9ca8263e>:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(df2[df1[name_id] == i][col]) > 0


In [12]:
result

,nreferencia,especificação,numero,local,titulo,produção,data,sistema,origem
0,"P.30.26/NT 1123,1124 - P. 31.27/NT 1125-1158",Professor Emérito. Concessão do Título de Professor Emérito a Octávio Ianni pela Congregação da Faculdade de Filosofia.,137.0,São Paulo/Salão Nobre- FFLCH,Professor Emérito.,CAPH,1997,Col/10 x 15 cm,Tabela49
1,"P.30.26/NT 1123,1124 - P. 31.27/NT 1125-1158",Professor Emérito. Concessão do Título de Professor Emérito a Octávio Ianni pela Congregação da Faculdade de Filosofia.,135.0,São Paulo/Salão Nobre- FFLCH,Professor Emérito.,CAPH,1997,Col/10 x 15 cm,GeralBD
2,117.6,Revolução Mexicana (Álbum). Villa e Zapata sentados na cadeira presidencial.,148.0,NaN,Revolução Mexicana (Álbum),NaN,NaN,"BP/18,0 x 24,0 cm",Tabela49
3,117.6,Revolução Mexicana (Álbum). Villa e Zapata sentados na cadeira presidencial.,147.0,NaN,Revolução Mexicana (Álbum),NaN,NaN,"BP/18,0 x 24,0 cm",GeralBD
4,Cx.2;E.2/NT 41,Revolução Mexicana: II - Representações artísticas - Murais - Muralista: Diego Rivera. História y perspectiva de México: la repressión.,197.0,NaN,"Revolução Mexicana: II - Representações artísticas - Murais, Gravuras.",NaN,NaN,"BP/15,5 x 22,5 cm",Tabela49
5,Cx.2;E.2/NT 41,Revolução Mexicana: II - Representações artísticas - Murais - Muralista: Diego Rivera. História y perspectiva de México: la repressión.,196.0,NaN,"Revolução Mexicana: II - Representações artísticas - Murais, Gravuras.",NaN,NaN,"BP/15,5 x 22,5 cm",GeralBD
6,Cx.2;E.1/NT 16,"Revolução Mexicana: I - Representações fotográficas. Venustiano Carranza, líder burguês da revolução. Fonte: Internacional.",172.0,NaN,Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,"BP/17,5 x 21,5 cm",Tabela49
7,Cx.2;E.1/NT 16,"Revolução Mexicana: I - Representações fotográficas. Venustiano Carranza, líder burguês da revolução. Fonte: Internacional.",171.0,NaN,Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,"BP/17,5 x 21,5 cm",GeralBD
8,Cx.2;E.1/NT 25,"Revolução Mexicana: I - Representações fotográficas. Sede da fazenda Soapayuca, no sul do México. Fonte Steinheiner-Pix.",181.0,NaN,Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,"BP/17,0 x 22,0 cm",Tabela49
9,Cx.2;E.1/NT 25,"Revolução Mexicana: I - Representações fotográficas. Sede da fazenda Soapayuca, no sul do México. Fonte Steinheiner-Pix.",180.0,NaN,Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,"BP/17,0 x 22,0 cm",GeralBD


In [13]:
len(result) /2 #77 registros estão em conflito

77.0

In [14]:
csv_exception = csv_exception.append(result) #adiciono estes registros para análise

In [15]:
csv_exception

,nreferencia,especificação,numero,ligação,sistema,titulo,local,data,produção,origem
200,NaN,Professor Emérito. Concessão do Título de Professor Emérito a Osvaldo Porchat pela Congregação da Faculdade de Filosofia.,137.0,5730.0,NaN,Professor Emérito,NaN,NaN,NaN,GeralBD
0,"P.30.26/NT 1123,1124 - P. 31.27/NT 1125-1158",Professor Emérito. Concessão do Título de Professor Emérito a Octávio Ianni pela Congregação da Faculdade de Filosofia.,137.0,NaN,Col/10 x 15 cm,Professor Emérito.,São Paulo/Salão Nobre- FFLCH,1997,CAPH,Tabela49
1,"P.30.26/NT 1123,1124 - P. 31.27/NT 1125-1158",Professor Emérito. Concessão do Título de Professor Emérito a Octávio Ianni pela Congregação da Faculdade de Filosofia.,135.0,NaN,Col/10 x 15 cm,Professor Emérito.,São Paulo/Salão Nobre- FFLCH,1997,CAPH,GeralBD
2,117.6,Revolução Mexicana (Álbum). Villa e Zapata sentados na cadeira presidencial.,148.0,NaN,"BP/18,0 x 24,0 cm",Revolução Mexicana (Álbum),NaN,NaN,NaN,Tabela49
3,117.6,Revolução Mexicana (Álbum). Villa e Zapata sentados na cadeira presidencial.,147.0,NaN,"BP/18,0 x 24,0 cm",Revolução Mexicana (Álbum),NaN,NaN,NaN,GeralBD
4,Cx.2;E.2/NT 41,Revolução Mexicana: II - Representações artísticas - Murais - Muralista: Diego Rivera. História y perspectiva de México: la repressión.,197.0,NaN,"BP/15,5 x 22,5 cm","Revolução Mexicana: II - Representações artísticas - Murais, Gravuras.",NaN,NaN,NaN,Tabela49
5,Cx.2;E.2/NT 41,Revolução Mexicana: II - Representações artísticas - Murais - Muralista: Diego Rivera. História y perspectiva de México: la repressión.,196.0,NaN,"BP/15,5 x 22,5 cm","Revolução Mexicana: II - Representações artísticas - Murais, Gravuras.",NaN,NaN,NaN,GeralBD
6,Cx.2;E.1/NT 16,"Revolução Mexicana: I - Representações fotográficas. Venustiano Carranza, líder burguês da revolução. Fonte: Internacional.",172.0,NaN,"BP/17,5 x 21,5 cm",Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,NaN,Tabela49
7,Cx.2;E.1/NT 16,"Revolução Mexicana: I - Representações fotográficas. Venustiano Carranza, líder burguês da revolução. Fonte: Internacional.",171.0,NaN,"BP/17,5 x 21,5 cm",Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,NaN,GeralBD
8,Cx.2;E.1/NT 25,"Revolução Mexicana: I - Representações fotográficas. Sede da fazenda Soapayuca, no sul do México. Fonte Steinheiner-Pix.",181.0,NaN,"BP/17,0 x 22,0 cm",Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,NaN,Tabela49


In [16]:
len(csv_exception)

155

In [17]:
print("Colunas que não são comuns: ",set(df_cat_foto.columns.values.tolist()) ^ set(df_bd_foto.columns.values.tolist()))

Colunas que não são comuns:  {'ligação', 'quantidade'}


df_cat_foto já tem todos os registros de df_bd_foto, falta apenas adicionar a coluna ligação à df_cat_foto (esta colunanão interessa para a database final**)

In [ ]:
"""
df_final_foto = df_cat_foto
df_final_foto= df_final_foto.assign(ligação='') #adiciona a coluna ligação
df_final_foto
"""
#df_final_foto.insert(len(csv_exception.columns),'ligação', '', True)

In [ ]:
"""
for index, row in df_bd_foto.T.iteritems():
    pos = df_final_foto.index[df_final_foto['nreferencia'] == row['nreferencia']] #pego a posição da row df_final
    df_final_foto.loc[pos.item(), 'ligação'] = row['ligação']
"""

In [18]:
df_final_foto = df_cat_foto
df_final_foto = df_final_foto.astype({'numero':'int32'}) #altero o tipo da col  para inteiro
#df_final_foto =  pd.to_numeric(df_final_foto['ligação'], errors='coerce')
#df = df.fillna(0)
#df_final_foto[ pd.to_numeric(df_final_foto['ligação'], errors='coerce').isnull()]
#df_final_foto = df_final_foto.astype({'ligação':'int32'}) #altero o tipo da col para inteiro

In [19]:
df_final_foto.dtypes

nreferencia      object
especificação    object
numero            int32
quantidade       object
sistema          object
titulo           object
local            object
data             object
produção         object
dtype: object

In [ ]:
#agr vamos analisar df_papel_foto
df_papel_foto

In [20]:
len(df_final_foto[df_final_foto['nreferencia'].isin(df_papel_foto.nreferencia.values.tolist())]) #as nrefencias não são iguais

0

In [21]:
print("Colunas que não são comuns: ",set(df_final_foto.columns.values.tolist()) ^ set(df_papel_foto.columns.values.tolist()))

Colunas que não são comuns:  {'ligação', 'numero', 'quantidade', 'produção', 'local'}


Apenas ligação é peculiar para df_papel_foto, como esta coluna não é interessante, basta adicionar os registros do papel fotográfico para df_final

In [23]:
df_final_foto = df_final_foto.append(df_papel_foto)

In [28]:
#vou verificar se não tem tombos repetidos
aux_duplicated = df_final_foto.nreferencia      
aux_duplicated.duplicated().any() #não tem

False

In [29]:
#tem que deletar os registros que separamos para análise
df_final_foto = df_final_foto[~df_final_foto['nreferencia'].isin(csv_exception.nreferencia.values.tolist()) ] 

In [32]:
len(df_final_foto)

150

In [40]:
df_final_foto = df_final_foto.rename(columns={'nreferencia': 'tombo'})
df_final_foto = df_final_foto.rename(columns={'titulo': 'title'})

In [49]:
#preciso verificar se não há tombo vazio e nem título
len(df_final_foto[df_final_foto['tombo'] == ''].index)
pd.isna(df_final_foto['tombo']).any() #false
np.where(pd.isnull(df_final_foto['tombo'])) #returned 0
len(df_final_foto['tombo'].unique()) == len(df_final_foto)
#ESTÁ TUDO CERTO COM A COLUNA TOMBO

True

In [54]:
#vou repetir o processo para title
len(df_final_foto[df_final_foto['title'] == ''].index)
pd.isna(df_final_foto['title']).any() #false
np.where(pd.isnull(df_final_foto['tombo'])) #returned 0
len(df_final_foto['tombo'].unique()) == len(df_final_foto)
#ESTÁ TUDO CERTO COM A COLUNA TITLE

True

In [56]:
len(df_final_foto) + len(csv_exception)

305

In [57]:
len(csv_exception)

155

Meu intervalo de nid será 17500 -  17800

In [67]:
nid = list(range(17500,17500+len(df_final_foto)))

In [71]:
df_final_foto['nid'] = nid
del df_final_foto['ligação']

In [83]:
columns_rename={
    'tombo': 'field_tombo_fotografia',
    'especificação': 'field_especificacao_fotografia',
    'numero': 'field_numero_fotografia',
    'quantidade': 'field_quantidade_fotografia',
    'sistema': 'field_sistema_fotografia',
    'local': 'field_local_fotografia',
    'data': 'field_data_fotografia',
    'produção': 'field_producao_fotografia'
    
}
df_final_foto = df_final_foto.rename(columns=columns_rename )

In [87]:
len(df_final_foto)

150

In [84]:
df_final_foto #PRONTO PARA EXPORTAÇÃO

,field_tombo_fotografia,field_especificacao_fotografia,field_numero_fotografia,field_quantidade_fotografia,field_sistema_fotografia,title,field_local_fotografia,field_data_fotografia,field_producao_fotografia,nid
19,115,"Álbum de fotografias sobre Guaratinguetá. (Áreas residenciais, comerciais, fabris, periféricas e ruínas históricas). [1945] Tamanhos: 6,5 x 9,0 cm / 7,5 x 5,0 cm / 10,5 x 8,0 cm",1.0,124,BP,Álbum de fotografias sobre Guaratinguetá.,NaN,1945,NaN,17500
20,Cx.1;E.1/NT 48,"Naturalistas - Jean Baptiste Debret (1768-1848). Florestas Virgens nas margens do Paraíba. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. II tomo, p.288 ss.)",75.0,1/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17501
21,Cx.1;E.1/NT 49,"Naturalistas - Jean Baptiste Debret (1768-1848). Vale na Serra do Mar. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. II tomo, p.296 ss.)",76.0,2/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17502
22,Cx.1;E.1/NT 50,"Naturalistas - Jean Baptiste Debret (1768-1848). Família de um chefe Camacã preparando-se para uma festa. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. v. I, pr 3.)",77.0,3/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17503
23,Cx.1;E.1/NT 51,"Naturalistas - Jean Baptiste Debret (1768-1848). Aldeia de caboclos em Cantagalo. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. I tomo, p.6.)",78.0,4/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17504
24,Cx.1;E.1/NT 52,"Naturalistas - Jean Baptiste Debret (1768-1848). Florestas virgens nas margens do Paraíba - Índios Guaianases. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. T. I pr. 13, T. II pr.1)",79.0,5/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17505
25,Cx.1;E.1/NT 53,"Naturalistas - Jean Baptiste Debret (1768-1848). Tribo Guaicuru em busca de novas pastagens. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. v. I, pr. 21)",80.0,6/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17506
26,Cx.1;E.1/NT 54,"Naturalistas - Jean Baptiste Debret (1768-1848). Um funcionário a passeio com sua família. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. v. II, pr. 5.)",79.0,7/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17507
27,Cx.1;E.1/NT 55,"Naturalistas - Jean Baptiste Debret (1768-1848). Uma senhora brasileira em seu lar. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. v. II p. 6.)",80.0,8/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17508
28,Cx.1;E.1/NT 56,"Naturalistas - Jean Baptiste Debret (1768-1848). Os refrescos no largo do Palácio. (Debret - Viagem Pitoresca e Histórica ao Brasil. Martins Ed. São Paulo, 1949. v. II, p. 9.)",81.0,9/12,"B/P 18,0 x 24,0 cm",Naturalistas - Jean Baptiste Debret (1768-1848).,NaN,NaN,NaN,17509


In [92]:
df_final_foto.groupby('field_producao_fotografia')['field_producao_fotografia'].agg([np.size])

,size
field_producao_fotografia,
CAPH,29
"Col/9 x 12,5 cm",1
Eduardo José Afonso,25
Pavan Nagae Associados,1


In [91]:
df_final_foto.groupby('field_local_fotografia')['field_local_fotografia'].agg([np.size])

,size
field_local_fotografia,
Anfiteatro de Convenções/USP,1
Anfiteatro de História/USP,2
Biblioteca do Depto.Geografia/USP,1
Bienal do Livro/SP,2
CAPH-USP,33
Departamento de Geografia/USP,2
Departamento de História/USP,2
Departamento de Letras/USP,1
IEB,1


In [90]:
df_final_foto.groupby('field_sistema_fotografia')['field_sistema_fotografia'].agg([np.size])

,size
field_sistema_fotografia,
"B/P 18,0 x 24,0 cm",67
"B/P 22,0 x 30,0 cm",25
BP,2
"BP/12,0 x 18,0 cm",1
"BP/18,0 x 24,0 cm",1
"BP/29,5 x 39,0 cm",5
Col/10 x 15 cm,5
"Col/10,0 x 15,0 cm",8
"Col/24,5 x 20,5",1


In [86]:
df_final_foto.to_csv("fotografia_final_2.csv", index=False, encoding='UTF-8' )

In [74]:
csv_exception

,nreferencia,especificação,numero,ligação,sistema,titulo,local,data,produção,origem
200,NaN,Professor Emérito. Concessão do Título de Professor Emérito a Osvaldo Porchat pela Congregação da Faculdade de Filosofia.,137.0,5730.0,NaN,Professor Emérito,NaN,NaN,NaN,GeralBD
0,"P.30.26/NT 1123,1124 - P. 31.27/NT 1125-1158",Professor Emérito. Concessão do Título de Professor Emérito a Octávio Ianni pela Congregação da Faculdade de Filosofia.,137.0,NaN,Col/10 x 15 cm,Professor Emérito.,São Paulo/Salão Nobre- FFLCH,1997,CAPH,Tabela49
1,"P.30.26/NT 1123,1124 - P. 31.27/NT 1125-1158",Professor Emérito. Concessão do Título de Professor Emérito a Octávio Ianni pela Congregação da Faculdade de Filosofia.,135.0,NaN,Col/10 x 15 cm,Professor Emérito.,São Paulo/Salão Nobre- FFLCH,1997,CAPH,GeralBD
2,117.6,Revolução Mexicana (Álbum). Villa e Zapata sentados na cadeira presidencial.,148.0,NaN,"BP/18,0 x 24,0 cm",Revolução Mexicana (Álbum),NaN,NaN,NaN,Tabela49
3,117.6,Revolução Mexicana (Álbum). Villa e Zapata sentados na cadeira presidencial.,147.0,NaN,"BP/18,0 x 24,0 cm",Revolução Mexicana (Álbum),NaN,NaN,NaN,GeralBD
4,Cx.2;E.2/NT 41,Revolução Mexicana: II - Representações artísticas - Murais - Muralista: Diego Rivera. História y perspectiva de México: la repressión.,197.0,NaN,"BP/15,5 x 22,5 cm","Revolução Mexicana: II - Representações artísticas - Murais, Gravuras.",NaN,NaN,NaN,Tabela49
5,Cx.2;E.2/NT 41,Revolução Mexicana: II - Representações artísticas - Murais - Muralista: Diego Rivera. História y perspectiva de México: la repressión.,196.0,NaN,"BP/15,5 x 22,5 cm","Revolução Mexicana: II - Representações artísticas - Murais, Gravuras.",NaN,NaN,NaN,GeralBD
6,Cx.2;E.1/NT 16,"Revolução Mexicana: I - Representações fotográficas. Venustiano Carranza, líder burguês da revolução. Fonte: Internacional.",172.0,NaN,"BP/17,5 x 21,5 cm",Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,NaN,Tabela49
7,Cx.2;E.1/NT 16,"Revolução Mexicana: I - Representações fotográficas. Venustiano Carranza, líder burguês da revolução. Fonte: Internacional.",171.0,NaN,"BP/17,5 x 21,5 cm",Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,NaN,GeralBD
8,Cx.2;E.1/NT 25,"Revolução Mexicana: I - Representações fotográficas. Sede da fazenda Soapayuca, no sul do México. Fonte Steinheiner-Pix.",181.0,NaN,"BP/17,0 x 22,0 cm",Revolução Mexicana: I - Representações fotográficas.,NaN,NaN,NaN,Tabela49


In [75]:
csv_exception.to_csv("fotografia_análise.csv", index=False, encoding='UTF-8' )

In [ ]:

#esboço da função compare_df

df1 = df_cat_foto
df2 = df_bd_foto
id1 =  set(df1['nreferencia'])
id2 =  set(df2['nreferencia'])
id_commom = list(id1.intersection(id2))
df1_name = '_tabela49'
df2_name = '_geralBD'
col_name = ['especificação','numero','local', 'titulo', 'produção', 'data', 'sistema']
diff = []
df_structure = {}
df_structure['nreferencia'] = []
for col in col_name:
    df_structure[col] = []
df_structure['origem'] = []

for i in id_commom:
    for col in col_name:
        if(
        len(df1[df1['nreferencia'] == i][col]) > 0
            and
        len(df2[df1['nreferencia'] == i][col]) > 0
        ):
            aux1 = df1[df1['nreferencia'] == i][col].item()
            aux2 = df2[df2['nreferencia'] == i][col].item()
            if(str(aux1) != 'nan'  or str(aux2) != 'nan'):
                if (col == 'numero' and aux1 < aux2):
                #if (aux1 != aux2):
                    for c in col_name:
                        aux = df_structure[c]
                        aux.append(df1[df1['nreferencia'] == i][c].item())
                        df_structure[c] = aux
                    aux = df_structure['nreferencia']
                    aux.append(i)
                    df_structure['nreferencia'] = aux
                    aux = df_structure['origem']
                    aux.append(df1_name)
                    df_structure['origem'] = aux
                    for c in col_name:
                        aux = df_structure[c]
                        aux.append(df2[df2['nreferencia'] == i][c].item())
                        df_structure[c] = aux
                    aux = df_structure['nreferencia']
                    aux.append(i)
                    df_structure['nreferencia'] = aux
                    aux = df_structure['origem']
                    aux.append(df2_name)
                    df_structure['origem'] = aux
                    
                    diff.append(df1[df1['nreferencia'] == i])
                    diff.append(df2[df2['nreferencia'] == i])
                    break
           